In [1]:
%load_ext autoreload
%autoreload 2
from stock_prediction.data_exploration.kaggle_etfs import load_etfs_data
from stock_prediction.commons import PANDAS_STYLE_VERTICAL_COLNAMES

# Rough Performance Analysis based on a Kaggle Scrapped dataset

The goal of this notebook is to short list a selection of tickers to include in our model building. 

Our goal is to develop a model that is able to predict how a given symbol will evolve, given its history/state and, possibly, information from the state of the market. Furthermore, we would like to bias towards a selection of symbols that perform well on long timescales. We envision an investor who is focusing on such symbols and wants to find a favourable time to invest on a few of them. Thus, below we look into an etfs dataset, for simplicity, and analyse a few metrics to short list the symbols we will work with.

**Disclaimer:** This analysis was done on this slightly outdated dataset for convenience. Since the goal is only to short list a set of supported symbols it should not affect the overall quality of the solution we will develop later (also because we focus on selecting on long term stable symbols).


In [2]:
df_etf_prices, df_etfs = load_etfs_data()

Loading ETFs data ...


## Performance by 10 years annualized return down to SPY

The SP500 index is regarded as one of the indices that repesents well the overall state of the market. Peforming as well as SP500 is often regarded as a "safe" way to "go with the market" and various metrics will use it as a baseline to measure gains/losses against the market (e.g., the Sharpe Ratio).

Thus, below we sort the available etf symbols on the dataset by 10 years annualized returns from the largest value down to SP500. We also display other metrics such as the 10 year volatility and sharpe ratio (which provides an indication of how significant the returns deviate relative to the market index given the volatility of the symbol).

In [14]:
window_str = "10years"

df_display = df_etfs.sort_values(by=f"fund_return_{window_str}", ascending=False)

spy_loc = df_display.index.get_loc(
    df_display[df_display["fund_symbol"] == "SPY"].iloc[-1].name
)

df_display.head(spy_loc + 1)[
    list(df_etfs.columns[0:1])
    +
    list(df_etfs.columns[9:10])
    + [
        "inception_date",
        "investment_type",
        "size_type",
        "avg_vol_3month",
        "fund_annual_report_net_expense_ratio",
        "fund_return_10years",
        f"fund_stdev_{window_str}",
        f"fund_sharpe_ratio_{window_str}",
    ]
].style.set_table_styles(PANDAS_STYLE_VERTICAL_COLNAMES).background_gradient(
    cmap="Greys"
).set_sticky(
    axis=1
)

,fund_symbol,exchange_name,inception_date,investment_type,size_type,avg_vol_3month,fund_annual_report_net_expense_ratio,fund_return_10years,fund_stdev_10years,fund_sharpe_ratio_10years
1853,TQQQ,NasdaqGM,2010-02-09,Growth,Large,41453804,0.009500,0.533000,50.080000,1.100000
1710,SOXL,NYSEArca,2010-03-11,Growth,Large,14050446,0.009900,0.504400,65.390000,0.960000
1816,TECL,NYSEArca,2008-12-17,Growth,Large,1377169,0.010100,0.483000,51.060000,1.030000
1538,RETL,NYSEArca,2010-07-14,Blend,Small,300142,0.010000,0.394800,72.240000,0.790000
1482,QLD,NYSEArca,2006-06-19,Growth,Large,3111084,0.009500,0.386700,32.420000,1.160000
1572,ROM,NYSEArca,2007-01-30,Growth,Large,75090,0.009500,0.376700,34.510000,1.090000
1912,USD,NYSEArca,2007-01-30,Growth,Large,193426,0.009500,0.368700,40.740000,0.970000
293,CURE,NYSEArca,2011-06-15,Value,Large,88516,0.010100,0.358400,40.490000,0.950000
1904,UPRO,NYSEArca,2009-06-23,Blend,Large,5753752,0.009300,0.333400,42.830000,0.890000
1876,UCC,NYSEArca,2007-01-30,Growth,Large,2590,0.009500,0.306700,30.810000,1.010000


In [4]:
spy_loc + 1

67

As of 2021, there were 67 ETFs in this datasets that were above or at SPY. We can round this up to the top 100 symbols and include them in our multivariate analysis.

In [16]:
df_display.head(100)[
    list(df_etfs.columns[0:1])]

,fund_symbol
1853,TQQQ
1710,SOXL
1816,TECL
1538,RETL
1482,QLD
...,...
1370,PEZ
786,FYC
780,FXO
2140,EUSA


## Scenarios for Investor
 
In this section, we look into several criteria to select ETFs to invest in from the short list above. Some investors will be more focused on the long term other on the short term. Thus, for the sake creating some scenarios of a user of the app we short list a few symbols per scenario.

### Performance 10y focused
- High sharpe ratio
- Returns >= SPY
- Long term STD close to SPY (< 16%)
- Avg_volum_3months > 1%SPY 

In [10]:
window_str = "10years"

df_display = df_etfs.sort_values(
    by=[f"fund_sharpe_ratio_{window_str}"], ascending=False
)

spy_loc = df_display.index.get_loc(
    df_display[df_display["fund_symbol"] == "SPY"].iloc[-1].name
)

df_display = df_display.head(spy_loc + 1)

df_display = df_display[
    df_display["avg_vol_3month"]
    >= (0.01 * df_display["avg_vol_3month"][df_display["fund_symbol"] == "SPY"].iloc[0])
]

df_display = df_display[df_display[f"fund_stdev_10years"] < 16]


df_display[
    list(df_etfs.columns[0:1])
    +
    list(df_etfs.columns[9:10])
    + [
        "inception_date",
        "investment_type",
        "size_type",
        "avg_vol_3month",
        "fund_annual_report_net_expense_ratio",
        "fund_return_10years",
        f"fund_stdev_10years",
        f"fund_sharpe_ratio_10years",
    ]
].style.set_table_styles(PANDAS_STYLE_VERTICAL_COLNAMES).background_gradient(
    cmap="Greys"
).set_sticky(
    axis=1
)

,fund_symbol,exchange_name,inception_date,investment_type,size_type,avg_vol_3month,fund_annual_report_net_expense_ratio,fund_return_10years,fund_stdev_10years,fund_sharpe_ratio_10years
1498,QQQ,NasdaqGM,1999-03-10,Growth,Large,46374673,0.002000,0.212500,15.680000,1.280000
2215,IHI,NYSEArca,2006-05-01,Growth,Large,939369,0.004200,0.189500,15.070000,1.190000
2240,IVW,NYSEArca,2000-05-22,Growth,Large,1835620,0.001800,0.170800,13.770000,1.180000
2244,IWF,NYSEArca,2000-05-22,Growth,Large,1707365,0.001900,0.176400,14.430000,1.160000
1762,SPY,NYSEArca,1993-01-22,Blend,Large,76940463,0.000940,0.147100,13.560000,1.040000


### Performance 5y focused
- High sharpe ratio
- Sharpe ratio > 1.1
- 5year STD  < 22%
- Avg_volum_3months > 1%SPY 

In [11]:
window_str = "5years"

df_display = df_etfs.sort_values(
    by=[f"fund_sharpe_ratio_{window_str}"], ascending=False
)

spy_loc = df_display.index.get_loc(
    df_display[df_display["fund_symbol"] == "SPY"].iloc[-1].name
)

df_display = df_display.head(spy_loc + 1)

df_display = df_display[
    df_display["avg_vol_3month"]
    >= (0.01 * df_display["avg_vol_3month"][df_display["fund_symbol"] == "SPY"].iloc[0])
]


df_display = df_display[df_display["fund_sharpe_ratio_3years"] >= 1.1]

df_display = df_display[df_display[f"fund_stdev_5years"] < 22]

df_display[
    list(df_etfs.columns[0:1])
    +
    list(df_etfs.columns[9:10])
    + [
        "inception_date",
        "investment_type",
        "size_type",
        "avg_vol_3month",
        "fund_annual_report_net_expense_ratio",
        "fund_return_5years",
        f"fund_stdev_5years",
        f"fund_sharpe_ratio_5years",
    ]
].style.set_table_styles(PANDAS_STYLE_VERTICAL_COLNAMES).background_gradient(
    cmap="Greys"
).set_sticky(
    axis=1
)

,fund_symbol,exchange_name,inception_date,investment_type,size_type,avg_vol_3month,fund_annual_report_net_expense_ratio,fund_return_5years,fund_stdev_5years,fund_sharpe_ratio_5years
1697,SMH,NasdaqGM,2011-12-20,Growth,Large,3470350,0.003500,0.374100,21.710000,1.530000
2212,IGV,BATS,2001-07-10,Growth,Large,1090682,0.004600,0.302200,18.180000,1.490000
1498,QQQ,NasdaqGM,1999-03-10,Growth,Large,46374673,0.002000,0.279800,16.890000,1.480000
2244,IWF,NYSEArca,2000-05-22,Growth,Large,1707365,0.001900,0.234200,16.100000,1.320000
2240,IVW,NYSEArca,2000-05-22,Growth,Large,1835620,0.001800,0.216100,15.290000,1.290000
2215,IHI,NYSEArca,2006-05-01,Growth,Large,939369,0.004200,0.220700,15.980000,1.260000
296,CWB,NYSEArca,2009-04-14,nan,nan,878659,0.004000,0.192500,14.700000,1.200000


### Performance 3y focused
- High sharpe ratio
- Returns >= 10%
- Sharpe ratio > 1.1
- Long term STD  < 30%
- Avg_volum_3months > 1%SPY 

In [12]:
window_str = "3years"

df_display = df_etfs.sort_values(
    by=[f"fund_sharpe_ratio_{window_str}"], ascending=False
)

spy_loc = df_display.index.get_loc(
    df_display[df_display["fund_symbol"] == "SPY"].iloc[-1].name
)
df_display = df_display.head(spy_loc + 1)

df_display = df_display[
    df_display["avg_vol_3month"]
    >= (0.01 * df_display["avg_vol_3month"][df_display["fund_symbol"] == "SPY"].iloc[0])
]


df_display = df_display[df_display["fund_sharpe_ratio_3years"] >= 1.1]

df_display = df_display[df_display[f"fund_stdev_3years"] < 30]
df_display = df_display[df_display[f"fund_return_3years"] > 0.1]


df_display[
    list(df_etfs.columns[0:1])
    +
    list(df_etfs.columns[9:10])
    + [
        "inception_date",
        "investment_type",
        "size_type",
        "avg_vol_3month",
        "fund_annual_report_net_expense_ratio",
        "fund_return_3years",
        f"fund_stdev_3years",
        f"fund_sharpe_ratio_3years",
    ]
].style.set_table_styles(PANDAS_STYLE_VERTICAL_COLNAMES).background_gradient(
    cmap="Greys"
).set_sticky(
    axis=1
)

,fund_symbol,exchange_name,inception_date,investment_type,size_type,avg_vol_3month,fund_annual_report_net_expense_ratio,fund_return_3years,fund_stdev_3years,fund_sharpe_ratio_3years
1697,SMH,NasdaqGM,2011-12-20,Growth,Large,3470350,0.003500,0.385700,24.660000,1.410000
1498,QQQ,NasdaqGM,1999-03-10,Growth,Large,46374673,0.002000,0.282900,20.240000,1.280000
2212,IGV,BATS,2001-07-10,Growth,Large,1090682,0.004600,0.290900,21.670000,1.240000
2244,IWF,NYSEArca,2000-05-22,Growth,Large,1707365,0.001900,0.248700,19.780000,1.160000
2215,IHI,NYSEArca,2006-05-01,Growth,Large,939369,0.004200,0.220000,17.970000,1.130000
2240,IVW,NYSEArca,2000-05-22,Growth,Large,1835620,0.001800,0.228800,18.730000,1.130000
296,CWB,NYSEArca,2009-04-14,nan,nan,878659,0.004000,0.223200,18.430000,1.120000
